
# Approach 

#### Clean and Transform Data

#### Classification Algorithms
    
    (1) Logit
    (2) Desicion Tree
    (3) Support Vector Machine
    
#### Train and validate Models
#### Out of Sample Comparison
    (1) False Negative Metric

# Main Conclusion
#### We have very small number of frauds => easy to overfit


#### Tree  outperfroms in out-of-sample:
- (1) simple logit 
- (2) SVM 

#### Tuning of SVM and tree is hard and tricky
#### Ensembling approach might be the best, i.e. random forest

In [3]:
Data_folder = r'DataFolder'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn import linear_model
from sklearn import svm

%matplotlib inline

In [4]:
df_fraud = pd.read_csv(Data_folder + 'fraud_prep.csv')

In [5]:
# Select label (for supervised learning)
df_fraud.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [6]:
print( "% of frauds is " + str( np.round( 100.*df_fraud.Class.value_counts()[1] / len(df_fraud), 2 ) ) )

% of frauds is 0.17


### Pre data cleaning:
-  Demean and standardized data

In [7]:
df_features = df_fraud.iloc[:,:-1].copy(deep = True)
mean = df_features.mean(axis = 0)
std = df_features.std(axis = 0)
df_features = df_features.sub (mean, axis = 1)
df_features = df_features.div(std, axis = 1)

- Drop string variables (nan are everywhere)

In [8]:
df_features = df_features.dropna(axis = 1)

# (1) Multiple Classification algo

In [9]:
X = df_features.copy(deep = True)
Y = df_fraud.Class

- Divide data into training, cross validation and out of sample

In [10]:
def train_validate_test_split(df, train_percent=.4, validate_percent=.2, seed=1):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

In [11]:
X_train, X_validate, X_test = train_validate_test_split(df_features)

# Find indexes for Y
Y_train = Y.copy(deep = True)
Y_validate = Y.copy(deep = True)
Y_test = Y.copy(deep = True)

Y_train = Y_train[X_train.index]
Y_validate = Y_validate[X_validate.index]
Y_test = Y_test[X_test.index]

In [12]:
print("All three sets are representative")
print( "% of frauds in train is " + str( np.round( 100.*Y_train.value_counts()[1] / len(Y_train), 2 ) ) )
print( "% of frauds in validation is " + str( np.round( 100.*Y_validate.value_counts()[1] / len(Y_validate), 2 ) ) )
print( "% of frauds in test is " + str( np.round( 100.*Y_test.value_counts()[1] / len(Y_test), 2 ) ) )

All three sets are representative
% of frauds in train is 0.16
% of frauds in validation is 0.18
% of frauds in test is 0.18


#### Train all model on train set

#### (a) Logit
- Logit requires an arbitrary chosen threhsold for classification
- Logit does not work well with False negative due to small number of frauds
- Logit works perfectly with False positive

In [13]:
logit = linear_model.LogisticRegression(C=1e5)
logit.fit(X_train, Y_train)
in_sample_fit = logit.predict(X_train )

In [16]:
# Transform logit into classifier: if prob > tau then fraud
threshold = 0.5
decision = 1* (in_sample_fit > threshold)

decision_df = pd.DataFrame(decision, columns={'Model'})
Data_df = pd.DataFrame( Y_train.values, columns = {"Data"})
Probability_df = decision_df.join(Data_df) # Column 0 = model, Column 1 = data

#### Main metrics of interests:

In [17]:
# When we have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] >0,:]   # data shows fraud
Prob = 1.*(tmp_df.iloc[:,0] == 0).sum()/ len(tmp_df)
print("Logit: False Negative " + str(Prob) )

# When we do not have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] ==0,:]   # data shows no fraud
Prob = 1.*(tmp_df.iloc[:,0] == 1).sum()/ len(tmp_df)
print("Logit: False Positive " + str(np.round(Prob,3)) )

Logit: False Negative 0.413978494624
Logit: False Positive 0.0


#### (b) Tree
- Tree does not require threhsold to make a decision
- Tree works perfectly, but it can be due to overfitting (see it later on out of sample data)

In [18]:
tree_model = tree.DecisionTreeClassifier()
tree_model.fit(X_train, Y_train)
decision = tree_model.predict(X_train)

In [19]:
decision_df = pd.DataFrame(decision, columns={'Model'})
Data_df = pd.DataFrame( Y_train.values, columns = {"Data"})
Probability_df = decision_df.join(Data_df) # Column 0 = model, Column 1 = data

#### Main metrics of interests:

In [20]:
# When we have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] >0,:]   # data shows fraud
Prob = 1.*(tmp_df.iloc[:,0] == 0).sum()/ len(tmp_df)
print("Tree: False Negative " + str(Prob) )

# When we do not have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] ==0,:]   # data shows no fraud
Prob = 1.*(tmp_df.iloc[:,0] == 1).sum()/ len(tmp_df)
print("Tree: False Positive " + str(np.round(Prob,3)) )

Tree: False Negative 0.0
Tree: False Positive 0.0


#### (c) SVM
- SVM does not require threhsold to make a decision
- SVM has problem with False negative; it is better than logit but worse than tree 

In [31]:
svm_model = svm.SVC()
svm_model.fit(X_train, Y_train)
decision = svm_model.predict(X_train)

In [32]:
decision_df = pd.DataFrame(decision, columns={'Model'})
Data_df = pd.DataFrame( Y_train.values, columns = {"Data"})
Probability_df = decision_df.join(Data_df) # Column 0 = model, Column 1 = data

#### Main metrics of interests:

In [35]:
# When we have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] >0,:]   # data shows fraud
Prob = 1.*(tmp_df.iloc[:,0] == 0).sum()/ len(tmp_df)
print("SVM: False Negative " + str(np.round(Prob,3) ))

# When we do not have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] ==0,:]   # data shows no fraud
Prob = 1.*(tmp_df.iloc[:,0] == 1).sum()/ len(tmp_df)
print("SVM: False Positive " + str(np.round(Prob,3)) )

SVM: False Negative 0.194
SVM: False Positive 0.0


# (2) - (3)
#### Out of Sample

#### Logit - out of sample
- Results are consistent with insample training

In [15]:
X_joint = X_train.append( X_validate) 
Y_joint = pd.DataFrame(np.hstack((Y_train.values, Y_validate.values)))

In [ ]:
logit = linear_model.LogisticRegression(C=1e5)
logit.fit(X_joint, Y_joint)
in_sample_fit = logit.predict(X_test )

In [51]:
# Transform logit into classifier: if prob > tau then fraud
threshold = 0.5
decision = 1* (in_sample_fit > threshold)

decision_df = pd.DataFrame(decision, columns={'Model'})
Data_df = pd.DataFrame( Y_test.values, columns = {"Data"})
Probability_df = decision_df.join(Data_df) # Column 0 = model, Column 1 = data

In [52]:
# When we have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] >0,:]   # data shows fraud
Prob = 1.*(tmp_df.iloc[:,0] == 0).sum()/ len(tmp_df)
print("Logit: False Negative " + str(Prob) )

# When we do not have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] ==0,:]   # data shows no fraud
Prob = 1.*(tmp_df.iloc[:,0] == 1).sum()/ len(tmp_df)
print("Logit: False Positive " + str(np.round(Prob,3)) )

Logit: False Negative 0.381188118812
Logit: False Positive 0.0


#### Tune Tree
- It requires a lot of running time
- Now, results are less impressive but still better than logit = > can work on more tuning (different parameters: depth of the tree, number of samples at internal nodes, maximum number of features)

In [60]:
min_samples_leaf_loop = [6,2,3] # Change number of samples at the external node
r_squared_loop = np.zeros(len(min_samples_leaf_loop))

for i in range(0, len(min_samples_leaf_loop)): # Loop over C
    print(i)
    #Train for fied C
    tree_model = tree.DecisionTreeClassifier(min_samples_leaf = min_samples_leaf_loop[i])
    tree_model.fit(X_train, Y_train)
    
    # Save loss function
    r_squared_loop[i] = tree_model.score(X_validate, Y_validate)

    
idx = np.argmax(r_squared_loop)
min_samples_leaf_optimal_tree = min_samples_leaf_loop[idx] 


0
1
2


In [54]:
# # Final result
tree_model = tree.DecisionTreeClassifier(min_samples_leaf = min_samples_leaf_optimal_tree)
tree_model.fit(X_train, Y_train)
decision = tree_model.predict(X_test)

decision_df = pd.DataFrame(decision, columns={'Model'})
Data_df = pd.DataFrame( Y_test.values, columns = {"Data"})
Probability_df = decision_df.join(Data_df) # Column 0 = model, Column 1 = data

In [55]:
# When we have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] >0,:]   # data shows fraud
Prob = 1.*(tmp_df.iloc[:,0] == 0).sum()/ len(tmp_df)
print("Tree: False Negative " + str(Prob) )

# When we do not have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] ==0,:]   # data shows no fraud
Prob = 1.*(tmp_df.iloc[:,0] == 1).sum()/ len(tmp_df)
print("Tree: False Positive " + str(np.round(Prob,3)) )

Tree: False Negative 0.252475247525
Tree: False Positive 0.0


#### Tune SVM
- It requires a lot of running time (the most time consuming process)
- Results are less impressive than in training sample
- Does not beat Tree, but is better than logit

In [56]:
C_loop = [1,2] # Change penalty
r_squared_loop = np.zeros(len(C_loop))

for i in range(0, len(C_loop)): # Loop over C
    print(i)
    # Train for fied C
    svm_model = svm.SVC(C = C_loop[i])
    svm_model.fit(X_train, Y_train)
    in_sample_fit = svm_model.predict(X_train)
    
    # Save loss function
    r_squared_loop[i] = svm_model.score(X_validate, Y_validate)

    
idx = np.argmax(r_squared_loop)
C_optimal_SVM = C_loop[idx]    

0
1


In [57]:
# # Final result
svm_model = svm.SVC(C = C_optimal_SVM)
svm_model.fit(X_train, Y_train)
decision = svm_model.predict(X_test)

decision_df = pd.DataFrame(decision, columns={'Model'})
Data_df = pd.DataFrame( Y_test.values, columns = {"Data"})
Probability_df = decision_df.join(Data_df) # Column 0 = model, Column 1 = data

In [58]:
# When we have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] >0,:]   # data shows fraud
Prob = 1.*(tmp_df.iloc[:,0] == 0).sum()/ len(tmp_df)
print("Tree: False Negative " + str(Prob) )

# When we do not have fraud in the data
tmp_df = Probability_df.copy(deep = True)
tmp_df = tmp_df.loc[ tmp_df.iloc[:,1] ==0,:]   # data shows no fraud
Prob = 1.*(tmp_df.iloc[:,0] == 1).sum()/ len(tmp_df)
print("Tree: False Positive " + str(np.round(Prob,3)) )

Tree: False Negative 0.331683168317
Tree: False Positive 0.0


# (4) Bonus:
- We should use mixture of models to identify frauduent transaction:

a) Use PCA to identify factor structure if it exists

b) Fit mixture for these PCA (otherwise dimension is too high for mixture model)

- Alternatively, clustering approach (i.e. k-nearest neighbors)